In [3]:
import pandas as pd
import warnings 
warnings.filterwarnings('ignore')


In [4]:
#payments = pd.read_csv('general_payments_2017.csv', chunksize=10000)

chunks = []
# Read the file in chunks
for chunk in pd.read_csv('general_payments_2017.csv', chunksize=10000):
    chunks.append(chunk)

# Combine all chunks into one dataframe
df = pd.concat(chunks, ignore_index=True)

# Display first few rows to see what we have
print(df.head())

  Change_Type       Covered_Recipient_Type  Teaching_Hospital_CCN  \
0   UNCHANGED  Covered Recipient Physician                    NaN   
1   UNCHANGED  Covered Recipient Physician                    NaN   
2   UNCHANGED  Covered Recipient Physician                    NaN   
3   UNCHANGED  Covered Recipient Physician                    NaN   
4   UNCHANGED  Covered Recipient Physician                    NaN   

   Teaching_Hospital_ID Teaching_Hospital_Name  Covered_Recipient_Profile_ID  \
0                   NaN                    NaN                        1642.0   
1                   NaN                    NaN                      804019.0   
2                   NaN                    NaN                     1258480.0   
3                   NaN                    NaN                      387229.0   
4                   NaN                    NaN                      112878.0   

   Covered_Recipient_NPI Covered_Recipient_First_Name  \
0           1.043281e+09                       

In [5]:
# Set display options to show all columns
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)
#df.head(n=5)
for i, col in enumerate(df.columns):
   print(f"{i}: {col}")

df.head(n=1)

# automation_prototype.ipynb


def create_unified_payment_view(df):
    # Convert 'Date_of_Payment' to datetime and extract 'Month_Year'
    df['Payment_Date'] = pd.to_datetime(df['Date_of_Payment'], errors='coerce')
    df['Month_Year'] = df['Payment_Date'].dt.strftime('%Y-%m')
    
    # Drop rows with invalid dates
    df = df.dropna(subset=['Payment_Date'])
    
    # Group by necessary fields and 'Nature_of_Payment_or_Transfer_of_Value'
    payment_breakdown = df.groupby([
        'Covered_Recipient_Profile_ID',
        'Covered_Recipient_First_Name',
        'Covered_Recipient_Last_Name',
        'Month_Year',
        'Nature_of_Payment_or_Transfer_of_Value'
    ], as_index=False).agg({
        'Total_Amount_of_Payment_USDollars': 'sum',
        'Number_of_Payments_Included_in_Total_Amount': 'sum'
    })
    
    # Create pivot table to reshape data
    pivot_table = payment_breakdown.pivot_table(
        index=[
            'Covered_Recipient_Profile_ID',
            'Covered_Recipient_First_Name',
            'Covered_Recipient_Last_Name',
            'Month_Year'
        ],
        columns='Nature_of_Payment_or_Transfer_of_Value',
        values=['Total_Amount_of_Payment_USDollars', 'Number_of_Payments_Included_in_Total_Amount'],
        aggfunc='sum',
        fill_value=0
    ).reset_index()
    
    # Flatten the multi-level column names
    pivot_table.columns = [
        f"{val}_{pay_type}".replace(' ', '_').replace(',', '')
        if pay_type else val
        for val, pay_type in pivot_table.columns
    ]
    
    # Calculate total amounts and payments per month
    monthly_totals = df.groupby([
        'Covered_Recipient_Profile_ID',
        'Covered_Recipient_First_Name',
        'Covered_Recipient_Last_Name',
        'Month_Year'
    ], as_index=False).agg({
        'Total_Amount_of_Payment_USDollars': 'sum',
        'Number_of_Payments_Included_in_Total_Amount': 'sum'
    }).rename(columns={
        'Total_Amount_of_Payment_USDollars': 'Total_Amount',
        'Number_of_Payments_Included_in_Total_Amount': 'Total_Payments'
    })
    
    # Merge pivot table and monthly totals
    merged_df = pd.merge(
        pivot_table,
        monthly_totals,
        on=[
            'Covered_Recipient_Profile_ID',
            'Covered_Recipient_First_Name',
            'Covered_Recipient_Last_Name',
            'Month_Year'
        ],
        how='left'
    )
    
    # Company breakdown
    company_breakdown = df.groupby([
        'Covered_Recipient_Profile_ID',
        'Covered_Recipient_First_Name',
        'Covered_Recipient_Last_Name',
        'Month_Year',
        'Applicable_Manufacturer_or_Applicable_GPO_Making_Payment_Name'
    ], as_index=False).agg({
        'Total_Amount_of_Payment_USDollars': 'sum',
        'Number_of_Payments_Included_in_Total_Amount': 'sum'
    }).rename(columns={
        'Applicable_Manufacturer_or_Applicable_GPO_Making_Payment_Name': 'Company',
        'Total_Amount_of_Payment_USDollars': 'Company_Amount',
        'Number_of_Payments_Included_in_Total_Amount': 'Company_Payments'
    })
    
    # Merge company data
    final_df = pd.merge(
        merged_df,
        company_breakdown,
        on=[
            'Covered_Recipient_Profile_ID',
            'Covered_Recipient_First_Name',
            'Covered_Recipient_Last_Name',
            'Month_Year'
        ],
        how='left'
    )
    
    # Rename columns to match desired output
    final_df.rename(columns={
        'Covered_Recipient_Profile_ID': 'Profile_ID',
        'Covered_Recipient_First_Name': 'First_Name',
        'Covered_Recipient_Last_Name': 'Last_Name',
        'Total_Amount_of_Payment_USDollars_Consulting_Fee': 'Consulting_Amount',
        'Number_of_Payments_Included_in_Total_Amount_Consulting_Fee': 'Consulting_Payments',
        'Total_Amount_of_Payment_USDollars_Food_and_Beverage': 'Food_Amount',
        'Number_of_Payments_Included_in_Total_Amount_Food_and_Beverage': 'Food_Payments',
        'Total_Amount_of_Payment_USDollars_Travel_and_Lodging': 'Travel_Amount',
        'Number_of_Payments_Included_in_Total_Amount_Travel_and_Lodging': 'Travel_Payments',
    }, inplace=True)
    
    # List of expected columns
    expected_columns = [
        'Profile_ID', 'First_Name', 'Last_Name', 'Month_Year',
        'Consulting_Amount', 'Consulting_Payments',
        'Food_Amount', 'Food_Payments',
        'Travel_Amount', 'Travel_Payments',
        'Total_Amount', 'Total_Payments',
        'Company', 'Company_Amount', 'Company_Payments'
    ]
    
    # Add missing columns with zeros if they don't exist
    for col in expected_columns:
        if col not in final_df.columns:
            final_df[col] = 0
    
    # Convert numeric columns to appropriate data types
    numeric_cols = [
        'Consulting_Amount', 'Consulting_Payments',
        'Food_Amount', 'Food_Payments',
        'Travel_Amount', 'Travel_Payments',
        'Total_Amount', 'Total_Payments',
        'Company_Amount', 'Company_Payments'
    ]
    
    for col in numeric_cols:
        final_df[col] = pd.to_numeric(final_df[col], errors='coerce').fillna(0)
    
    # Select and reorder columns
    final_df = final_df[expected_columns]
    
    # Sort and reset index
    final_df.sort_values(by=['Profile_ID', 'Month_Year'], inplace=True)
    final_df.reset_index(drop=True, inplace=True)
    
    return final_df

# Create the unified payment view
unified_view = create_unified_payment_view(df)


unified_view.head(n=20)


0: Change_Type
1: Covered_Recipient_Type
2: Teaching_Hospital_CCN
3: Teaching_Hospital_ID
4: Teaching_Hospital_Name
5: Covered_Recipient_Profile_ID
6: Covered_Recipient_NPI
7: Covered_Recipient_First_Name
8: Covered_Recipient_Middle_Name
9: Covered_Recipient_Last_Name
10: Covered_Recipient_Name_Suffix
11: Recipient_Primary_Business_Street_Address_Line1
12: Recipient_Primary_Business_Street_Address_Line2
13: Recipient_City
14: Recipient_State
15: Recipient_Zip_Code
16: Recipient_Country
17: Recipient_Province
18: Recipient_Postal_Code
19: Covered_Recipient_Primary_Type_1
20: Covered_Recipient_Primary_Type_2
21: Covered_Recipient_Primary_Type_3
22: Covered_Recipient_Primary_Type_4
23: Covered_Recipient_Primary_Type_5
24: Covered_Recipient_Primary_Type_6
25: Covered_Recipient_Specialty_1
26: Covered_Recipient_Specialty_2
27: Covered_Recipient_Specialty_3
28: Covered_Recipient_Specialty_4
29: Covered_Recipient_Specialty_5
30: Covered_Recipient_Specialty_6
31: Covered_Recipient_License_Stat

,Profile_ID,First_Name,Last_Name,Month_Year,Consulting_Amount,Consulting_Payments,Food_Amount,Food_Payments,Travel_Amount,Travel_Payments,Total_Amount,Total_Payments,Company,Company_Amount,Company_Payments
0,1.0,BARRY,SIMMONS,2017-02,0.00,0,0.00,0,0.00,0,553.85,8,"A-dec, Inc.",553.85,8
1,1.0,BARRON,SIMMONS,2017-10,0.00,0,0.00,0,0.00,0,328.19,1,"DentalEZ, Inc.",328.19,1
2,1.0,BARRY,SIMMONS,2017-10,123.25,1,0.00,0,0.00,0,123.25,1,Dentsply LLC,123.25,1
3,2.0,Lindsey,Schuster,2017-01,0.00,0,16.19,1,0.00,0,16.19,1,AstraZeneca Pharmaceuticals LP,16.19,1
4,2.0,LINDSEY,SCHUSTER,2017-03,0.00,0,133.07,3,0.00,0,133.07,3,Merck Sharp & Dohme Corporation,131.21,2
5,2.0,LINDSEY,SCHUSTER,2017-03,0.00,0,133.07,3,0.00,0,133.07,3,PFIZER INC.,1.86,1
6,2.0,LINDSEY,SCHUSTER,2017-04,0.00,0,11.19,1,0.00,0,11.19,1,Allergan Inc.,11.19,1
7,2.0,LINDSEY,SCHUSTER,2017-07,0.00,0,16.50,1,0.00,0,16.50,1,Merck Sharp & Dohme Corporation,16.50,1
8,2.0,LINDSEY,SCHUSTER,2017-10,0.00,0,38.04,3,0.00,0,38.04,3,Merck Sharp & Dohme Corporation,36.30,2
9,2.0,LINDSEY,SCHUSTER,2017-10,0.00,0,38.04,3,0.00,0,38.04,3,PFIZER INC.,1.74,1


In [9]:
unified_view.to_csv('unified_view.csv', index=False)

,Covered_Recipient_Profile_ID,Covered_Recipient_First_Name,Covered_Recipient_Last_Name,Nature_of_Payment_or_Transfer_of_Value,Number_of_Payments_Included_in_Total_Amount,Total_Amount_of_Payment_USDollars
0,1.0,BARRON,SIMMONS,[Gift],1,328.19
1,1.0,BARRY,SIMMONS,"[Consulting Fee, Gift]",9,677.10
2,2.0,LINDSEY,SCHUSTER,[Food and Beverage],10,380.91
3,2.0,Lindsey,Schuster,[Food and Beverage],1,16.19
4,3.0,JUSTIN,RACKLEY,[Food and Beverage],6,107.10
5,4.0,Lisa,Hamaker,[Food and Beverage],1,75.27
6,5.0,PATRICK,DAVOL,"[Food and Beverage, Travel and Lodging]",9,1218.72
7,6.0,VIJAYA,MUMMADI,"[Food and Beverage, Education]",76,1065.30
8,6.0,Vijaya,Mummadi,"[Food and Beverage, Education]",25,299.55
9,7.0,JILL,WIRTH,"[Food and Beverage, Education]",134,1589.98


In [7]:
# Create pivot table with both amounts and counts

# Group by physician ID and nature of payment
# Group by physician info and nature of payment
# Create the basic groupby structure
payment_analysis = df.groupby([
    'Covered_Recipient_Profile_ID',
    'Covered_Recipient_First_Name',
    'Covered_Recipient_Last_Name'
]).agg({
    'Nature_of_Payment_or_Transfer_of_Value': lambda x: list(x.unique()),  # Get unique payment types
    'Number_of_Payments_Included_in_Total_Amount': 'sum',
    'Total_Amount_of_Payment_USDollars': 'sum'
}).reset_index()

payment_analysis.head(n=20)
payment
payment_analysis = pd.pivot_table(df,
    index=['Covered_Recipient_Profile_ID', 'Covered_Recipient_First_Name', 'Covered_Recipient_Last_Name'],
    columns='Nature_of_Payment_or_Transfer_of_Value',
    values=['Total_Amount_of_Payment_USDollars', 'Number_of_Payments_Included_in_Total_Amount'],
    aggfunc='sum',
    fill_value=0
).reset_index()

gebhart_data = payment_analysis[
    (payment_analysis['Covered_Recipient_First_Name'].str.contains('Jeremy', case=False, na=False)) & 
    (payment_analysis['Covered_Recipient_Last_Name'].str.contains('Gebhart', case=False, na=False))
]

print(gebhart_data)

payment_analysis.head()

Empty DataFrame
Columns: [(Covered_Recipient_Profile_ID, ), (Covered_Recipient_First_Name, ), (Covered_Recipient_Last_Name, ), (Number_of_Payments_Included_in_Total_Amount, Charitable Contribution), (Number_of_Payments_Included_in_Total_Amount, Compensation for services other than consulting, including serving as faculty or as a speaker at a venue other than a continuing education program), (Number_of_Payments_Included_in_Total_Amount, Compensation for serving as faculty or as a speaker for a non-accredited and noncertified continuing education program), (Number_of_Payments_Included_in_Total_Amount, Compensation for serving as faculty or as a speaker for an accredited or certified continuing education program), (Number_of_Payments_Included_in_Total_Amount, Consulting Fee), (Number_of_Payments_Included_in_Total_Amount, Current or prospective ownership or investment interest), (Number_of_Payments_Included_in_Total_Amount, Education), (Number_of_Payments_Included_in_Total_Amount, Entertai

Covered_Recipient_Profile_ID  \
Nature_of_Payment_or_Transfer_of_Value                                
0                                                               1.0   
1                                                               1.0   
2                                                               2.0   
3                                                               2.0   
4                                                               3.0   

                                       Covered_Recipient_First_Name  \
Nature_of_Payment_or_Transfer_of_Value                                
0                                                            BARRON   
1                                                             BARRY   
2                                                           LINDSEY   
3                                                           Lindsey   
4                                                            JUSTIN   

                                       Covered_Recipient_Last_Name  \
Nature_of_Payment_or_Transfer_of_Value                               
0                                                          SIMMONS   
1                                                          SIMMONS   
2                                                         SCHUSTER   
3                                                         Schuster   
4                                                          RACKLEY   

                                       Number_of_Payments_Included_in_Total_Amount  \
Nature_of_Payment_or_Transfer_of_Value                     Charitable Contribution   
0                                                                                0   
1                                                                                0   
2                                                                                0   
3                                                                                0   
4                                                                                0   

                                                                                                                                                                                           \
Nature_of_Payment_or_Transfer_of_Value Compensation for services other than consulting, including serving as faculty or as a speaker at a venue other than a continuing education program   
0                                                                                                                                                                                       0   
1                                                                                                                                                                                       0   
2                                                                                                                                                                                       0   
3                                                                                                                                                                                       0   
4                                                                                                                                                                                       0   

                                                                                                                                                               \
Nature_of_Payment_or_Transfer_of_Value Compensation for serving as faculty or as a speaker for a non-accredited and noncertified continuing education program   
0                                                                                                                                                           0   
1                                                                                                                                             

In [28]:
physician_list = [
    "Jonathan M Vigdorchik", "Abhinav K Sharma", "Aaron J Buckland", "Ameer M Elbuluk", 
    "David J Mayman", "Alex J Anatone", "Mark P Figgie", "William J Long", "Peter K Sculco",
    "Justin A Magnuson", "Gregory R Toci", "Andrew M Star", "Brian C Chung", "Michael Stefl",
    "Raymond J Hah", "Jeffrey C Wang", "Nathanael D Heckmann", "Brandon R Bukowski",
    "Nicholas J Clark", "Brett A Freedman", "Daniel J Berry", "Matthew P Abdel", "Peter Zhou",
    "Themistocles S Protopsaltis", "Peter G Passias", "Michael S Pheasant", "Edward M DelSole",
    "John J Mercuri", "Lucas Nikkel", "Jason M Jennings", "Prem N Ramkumar", "Antonia F Chen",
    "Richard Iorio", "Jeffrey K Lange", "Emily S Mills", "Assem A Sultan", "Anton Khlopas",
    "Nicolas S Piuzzi", "Morad Chughtai", "Michael A Mont", "Hector Castillo", "Lewis L Shi",
    "Michael J Lee", "Mostafa El Dafrawy", "Pradip Ramamurti", "Nicole D Quinlan", 
    "Matthew J Deasey", "Brian C Werner", "James A Browne", "Nicholas A Trasolini",
    "William G Lundergan", "Braden McKnight", "Rajan Murgai", "Adam E Roy", "Charles S Carrier",
    "Thomas S Thornhill", "Jeffrey N Katz", "Drake G LeBrun", "Nathaniel T Ondeck",
    "Robert C Marchand", "Geoffrey H Westrich", "Andrew M Schwartz", "Samuel S Wellman",
    "Michael P Bolognesi", "Sean P Ryan", "Aidin E Pour", "Jordan H Green",
    "Thomas H Christensen", "Jay R Lieberman", "Jeffrey J Barry", "David C Sing",
    "Thomas P Vail", "Erik N Hansen", "Andrew T Garber", "John R Dimar", 
    "Steven D Glassman", "Adam R Cochran", "Deeptee Jain", "Elizabeth Lord",
    "Joseph D Zuckerman", "Christopher N Carender", "David E DeMik", "Sean Slaven",
    "Andrew J Pugely", "John Callaghan", "Hyunwoo Kang", "James I Huddleston",
    "Zachary C Lum", "John P Meehan", "Daniel J Blizzard", "Christopher R Brown",
    "Srikanth Divi", "Alexander R Vaccaro", "William J Hozack", 
    "Alejandro Gonzalez Della Valle", "Richard B Han", "Gerald Andah", "Eric Hume",
    "Kristen Radcliff", "Fabio Orozco", "Eric Chen", "Zachary Post", "Alvin Ong",
    "Matthew J Grosso", "Arjun Saxena", "Kevin I Perry", "Alan Daniels",
    "Vijay J Rasquinha", "Sreevathsa Boraiah", "Stephen R Chen", "Christopher J Como",
    "Brandon K Couch", "Brian A Klatt", "William F Donaldson", "Joon Y Lee",
    "Jeremy D Shaw", "Ryan Sutton", "Javad Parvizi", "Armin Arshi", "Yale A Fillingham",
    "Mark J Lambrechts", "Aditya Mazmudar", "Yunsoo Lee", "Karan Goswami",
    "Brian A Karamian", "Jose A Canseco", "Alan S Hilibrand", "Christopher K Kepler",
    "Gregory D Schroeder", "James J Purtill", "Ritesh R Shah", "Alexander C Gordon",
    "Steven M Mardjetko", "Wayne M Goldstein", "Jeffrey M Goldstein", "Laviel Fernandez",
    "Russell J Bodner", "Kevin P Smidt", "Tyler A Luthringer", "Joseph M Nessler",
    "Richard L Illgen", "Daniel N Bracey", "Vishal Hegde", "Arthur L Malkani",
    "Shikha Sachdeva", "Joseph P Nessler", "Steven F Harwin", "Malynda S Wynn",
    "Nicholas A Bedard", "Jesse Otero", "Timothy S Brown", "Justin K Fritz",
    "Bradford S Waddell", "Kurt J Kitziger", "Paul C Peters", "Brian P Gladnick",
    "Omar A Behery", "Stephen G George", "Shaleen Vira", "Joshua R Labott",
    "Cody C Wyles", "Daniel Alsoof", "Christopher L McDonald", "Bassel G Diebo",
    "Eren O Kuris", "Valentin Antoci", "Thomas J Errico", "Gregory T Minutillo",
    "Caroline B Granruth", "Jaret M Karnuta", "Charles L Nelson", "Shyam A Patel",
    "Daniel B C Reid", "Kevin J Disilvestro", "Jacob M Babu", "Hayden N Box",
    "David W Hennessy", "Timothy L Kahn", "Jeremy M Gililland", "Frank J Schwab",
    "Gregory M Mundis", "William G Rainer", "Michael J Taunton", "Michael Ransone",
    "Keith Fehring", "Thomas Fehring", "Patrick Kellam", "Christopher Pelt",
    "Christopher L Peters", "Patrick Cahill", "Paul D Sponseller", "Jenna Bernstein",
    "Ryan Charette", "Matthew Sloan", "Joshua P Rainey", "Jeffrey J Frandsen",
    "Lucas A Anderson", "Breana R Siljander", "David W Polly", "David G Deckey",
    "Christian S Rosenow", "Cara Lai", "Zachary K Christopher", "Jens T Verhey",
    "Adam J Schwartz", "Joshua S Bingham", "Karim A Shafi", "Adam M Wegner",
    "Brian P Chalmers", "Thomas P Sculco", "Jacob M Elkins", "Todd M Miner",
    "Charlie C Yang", "Douglas A Dennis", "Tarek A Taha", "Christopher Bejcek",
    "Jorge De Leon", "Farhan Ahmad", "Kavina Patel", "Frank Buttacavoli",
    "Georges Bounajem", "Michael C Mariorenzi", "Dominic T Kleinhenz", "Eric M Cohen",
    "Ran S Schwarzkopf", "Nima Eftekhary", "Colin T Penrose", "Abiram Bala",
    "Thorsten M Seyler", "Jonathan L Berliner", "Douglas E Padgett", "Seth A Jerabek",
    "Paul S Roettges", "Jordan L Smith", "John T Ruth", "Doug S Weinberg",
    "Jeremy J Gebhart", "Raymond W Liu", "Colin A Mudrick", "J Stuart Melvin",
    "Bryan D Springer", "Vaibhav Kanawade", "Lawrence D Dorr", "Zhinian Wan",
    "John C Clohisy"
]

# Create pivot table
payment_analysis = pd.pivot_table(df,
    index=['Covered_Recipient_Profile_ID', 'Covered_Recipient_First_Name', 'Covered_Recipient_Last_Name'],
    columns='Nature_of_Payment_or_Transfer_of_Value',
    values=['Total_Amount_of_Payment_USDollars', 'Number_of_Payments_Included_in_Total_Amount'],
    aggfunc='sum',
    fill_value=0
).reset_index()

# Create empty list to store results
results = []

# Loop through each name in our specific list
for full_name in physician_list:
    name_parts = full_name.split()
    first_name = name_parts[0]
    last_name = name_parts[-1]
    
    # Query the pivot table
    if not physician_data.empty:
        # Convert names to uppercase for consistent grouping
        physician_data['Covered_Recipient_First_Name'] = physician_data['Covered_Recipient_First_Name'].str.upper()
        physician_data['Covered_Recipient_Last_Name'] = physician_data['Covered_Recipient_Last_Name'].str.upper()
        
        # Group by ID (this will combine entries with same ID regardless of original case)
        physician_data = physician_data.groupby('Covered_Recipient_Profile_ID').first().reset_index()
        results.append(physician_data)
    
    if not physician_data.empty:
        results.append(physician_data)

# Combine all results
final_results = pd.concat(results, ignore_index=True)

for full_name in physician_list:
    name_parts = full_name.split()
    first_name = name_parts[0]
    last_name = name_parts[-1]
    
    # Query the pivot table
    physician_data = payment_analysis[
        (payment_analysis['Covered_Recipient_First_Name'].str.contains(first_name, case=False, na=False)) & 
        (payment_analysis['Covered_Recipient_Last_Name'].str.contains(last_name, case=False, na=False))
    ]
    
    if not physician_data.empty:
        # Convert names to uppercase for consistent grouping
        physician_data['Covered_Recipient_First_Name'] = physician_data['Covered_Recipient_First_Name'].str.upper()
        physician_data['Covered_Recipient_Last_Name'] = physician_data['Covered_Recipient_Last_Name'].str.upper()

        # Flatten MultiIndex columns
        physician_data.columns = ['_'.join(col).strip('_') if isinstance(col, tuple) else col 
                                  for col in physician_data.columns]
        
        # Append to final results
        final_results = pd.concat([final_results, physician_data], ignore_index=True)
        
# **After collecting all data, group by ID to remove duplicates**
final_results = final_results.groupby('Covered_Recipient_Profile_ID', as_index=False).agg({
    'Covered_Recipient_First_Name': 'first',
    'Covered_Recipient_Last_Name': 'first',
    **{col: 'sum' for col in final_results.columns if col not in [
        'Covered_Recipient_Profile_ID', 'Covered_Recipient_First_Name', 'Covered_Recipient_Last_Name']}
})

# Save to CSV
final_results.to_csv('complete_physician_list_payments.csv', index=False)

print(f"Processed {len(physician_list)} names")
print(f"Found payment data for {len(final_results)} physicians")
print("\nFirst few entries:")
print(final_results.head())

Processed 245 names
Found payment data for 672 physicians

First few entries:
   Covered_Recipient_Profile_ID Covered_Recipient_First_Name  \
0                        1191.0                    FREDERICK   
1                        1214.0                      MICHAEL   
2                        5483.0                      RICHARD   
3                        8487.0                      MICHAEL   
4                       11063.0                      WILLIAM   

  Covered_Recipient_Last_Name  (Covered_Recipient_Profile_ID, )  \
0                       COHEN                               0.0   
1                         LEE                               0.0   
2                       CHANG                               0.0   
3                     MONTESI                               0.0   
4                   DONALDSON                               0.0   

  (Covered_Recipient_First_Name, ) (Covered_Recipient_Last_Name, )  \
0                                0                              

In [22]:
final_results.head()
final_results.to_csv('complete_physician_list_payments.csv', index=False)